In [15]:
import pickle

def read_pickle_file(file_path):
    """Reads a pickle file and returns its contents.

    Args:
        file_path (str): The path to the pickle file.

    Returns:
        The contents of the pickle file.
    """

    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        return data
    except FileNotFoundError:
        print(f"Error: Pickle file '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error reading pickle file: {e}")
        return None

# Example usage:
file_path = r"C:\Users\nisha\Documents\PythonProjects\TradingAnalysis\src\data\pickle\historical_data_minute.pkl"
data = read_pickle_file(file_path)
#symbol='AAPL'
#data=data[data['symbol']==symbol]

if data is not None:
    # Process the data
    print(f'Number of rows in dataset is {len(data)}')
else:
    print("Failed to read the pickle file.")

Number of rows in dataset is 5705


In [16]:
data['symbol'].value_counts()

symbol
MU     2912
AMD    2793
Name: count, dtype: int64

In [17]:
data.head(5)

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2024-10-01 08:00:00+00:00,164.08,164.12,163.81,163.81,1562.0,29.0,163.973905,AMD
2024-10-01 08:01:00+00:00,163.77,163.77,163.77,163.77,721.0,17.0,163.829792,AMD
2024-10-01 08:02:00+00:00,164.00,164.00,164.00,164.00,115.0,6.0,164.000522,AMD
2024-10-01 08:04:00+00:00,164.02,164.02,164.02,164.02,141.0,8.0,164.042979,AMD
2024-10-01 08:08:00+00:00,164.10,164.10,164.10,164.10,796.0,19.0,164.099749,AMD
